In [1]:
#!/usr/bin/python3

In [2]:
# to use 
# do <pip install Faker> in your activated dev env 

# Data Pre-processing 
The NYC housing data used for this project was sourced from https://www.kaggle.com/datasets/kaijiechen/nyc-housing-data-20032019?resource=download

Customer data was created by the members of this group to simulate real world customer data.


In [71]:
# import python libraries
from faker import Faker
import random_address
import pandas as pd 
import datetime
from pathlib import Path
import requests
from collections import defaultdict
 

# Data preparation
Creating dummy customer data 

In [72]:
# create variables (list) 
first_names = []
last_names = []
jobs = []
credit_scores = []
address = []
year1 = []
year2 = []
year3 = []
dob = []
ssn = []
current_debt = []

# create customer data 
faker = Faker()

for i in range(30):
    first_names.append(faker.first_name())
    last_names.append(faker.last_name())
    jobs.append((faker.job()))
    credit_scores.append(faker.random_int(585, 790))
    address.append(faker.address())
    year1.append(faker.random_int(98000, 220000))
    year2.append(faker.random_int(120000, 370000))
    year3.append(faker.random_int(135000, 450000))
    ssn.append(str(faker.random_int(135, 654)) + '-' + str(faker.random_int(23, 88)) + '-' + str(faker.random_int(2234, 7856)))
    current_debt.append(faker.random_int(4500, 62000))
    

In [73]:
# create an empty dob list 
dob = []

start_date = datetime.date(year=1974, month=1, day=1)

for i in range(20):
    # fake.date_between(start_date='today', end_date='+30y')
    dob.append(str(faker.date_between(start_date=start_date, end_date='-32y')))
    
for i in range(10):
    dob.append(str(faker.date_between(start_date=start_date, end_date='-42y')))
for i in range(10):
    dob.append(str(faker.date_between(start_date=start_date, end_date='-45y')))

In [74]:

df = pd.DataFrame(list(zip(first_names, last_names, jobs, credit_scores, address, year1, year2, year3, dob, ssn, current_debt)),
               columns =['first_name', 'last_names', 'jobs', 'credit_scores', 'address', 'year1', 'year2', 'year3', \
                        'dob', 'ssn', 'current_debt'])

In [75]:
# convert the dob column to datetime object 
df['dob'] = pd.to_datetime(df['dob'])

# subtract 2022 from the year(dob) to get the age of the customer
df['age'] =  df['dob'].apply(lambda x: 2022 - int(x.year))

In [76]:
# checking df datatypes
df.dtypes

first_name               object
last_names               object
jobs                     object
credit_scores             int64
address                  object
year1                     int64
year2                     int64
year3                     int64
dob              datetime64[ns]
ssn                      object
current_debt              int64
age                       int64
dtype: object

In [77]:
# removing \n from address 
def remove_newline(strng):
    """
    function to remove \n from a string
    
    input: string strng
    output: new strng
    
    """
    return strng.replace('\n', ' ')

In [78]:
# apply function to address column
df['address'] = df['address'].apply(remove_newline)

In [79]:
df.head()

,first_name,last_names,jobs,credit_scores,address,year1,year2,year3,dob,ssn,current_debt,age
0,Matthew,Roberts,Forest/woodland manager,700,"587 Richard Mills Lake Phyllisstad, MS 72339",207290,327128,209558,1978-10-24,556-83-4105,13719,44
1,Kimberly,Bradley,Retail merchandiser,590,"2302 Rhonda Drive Suite 301 Jameshaven, KS 43582",138768,154584,432014,1989-04-01,506-29-6214,28299,33
2,George,Francis,Data processing manager,632,"2388 Campos Plain Suite 538 North Mallory, HI ...",138151,270091,377325,1978-08-14,634-87-2939,24983,44
3,Kristina,Guerra,Multimedia programmer,642,"985 Erika Cape South Paul, GA 42964",163500,342981,242466,1986-07-18,372-35-6905,40928,36
4,Jeff,Cummings,Personal assistant,768,"59065 Gina Garden Suite 292 New Christopher, M...",101206,176421,147224,1977-01-09,485-41-5033,27560,45


In [13]:
# write file to resouces folder 
df.to_csv('Resources/customer_data.csv', index=False)

## Part 2:
## Cleaning the NYC housing data

New York Housing Data: Data preparation. 
To get the nyc_housing_data_2013_2019.csv file please download from this link https://www.kaggle.com/datasets/kaijiechen/nyc-housing-data-20032019?resource=download

In [14]:
# read in df 
# update the file path to the location of your downloaded file
df_kaggle = pd.read_csv(Path("<path_to_data.csv>"))

In [15]:
df_kaggle.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009.0,2134.0,3542.0,1899.0,1800000,2003-01-22 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009.0,5746.0,2700.0,1900.0,0,2003-12-18 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009.0,5746.0,2700.0,1900.0,0,2003-12-18 00:00:00


In [16]:
df_kaggle.dtypes

BOROUGH                      int64
NEIGHBORHOOD                object
BUILDING CLASS CATEGORY     object
ADDRESS                     object
ZIP CODE                   float64
LAND SQUARE FEET           float64
GROSS SQUARE FEET          float64
YEAR BUILT                 float64
SALE PRICE                   int64
SALE DATE                   object
dtype: object

In [17]:
# convert ADDRESS, ZIP CODE, LAND SQUARE FEET, GROSS SQUARE FEET

df_kaggle['ZIP CODE'] = pd.to_numeric(df_kaggle['ZIP CODE'], errors='coerce').astype('Int64')

In [18]:
df_kaggle['LAND SQUARE FEET'] = pd.to_numeric(df_kaggle['LAND SQUARE FEET'], errors='coerce').astype('Int64')

In [19]:
df_kaggle['GROSS SQUARE FEET'] = pd.to_numeric(df_kaggle['GROSS SQUARE FEET'], errors='coerce').astype('Int64')

In [20]:
df_kaggle['YEAR BUILT'] = pd.to_numeric(df_kaggle['YEAR BUILT'], errors='coerce').astype('Int64')

In [21]:
df_kaggle.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899,1800000,2003-01-22 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009,5746,2700,1900,0,2003-12-18 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009,5746,2700,1900,0,2003-12-18 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,125 AVENUE D,10009,2185,5725,1910,426000,2003-10-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,610 EAST 9 STREET,10009,2543,11568,1910,0,2003-02-28 00:00:00


In [22]:
# remove any houses where sales price is less than 300000

df_kaggle_new = df_kaggle[df_kaggle['SALE PRICE'] > 300000]

In [23]:
df_kaggle_new.head(4)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899,1800000,2003-01-22 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,125 AVENUE D,10009,2185,5725,1910,426000,2003-10-23 00:00:00
11,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,14 AVENUE A,10009,2377,8722,1900,1600000,2003-09-08 00:00:00
12,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,536 EAST 6 STREET,10009,1957,6200,1920,1060000,2003-04-30 00:00:00


In [24]:
df_kaggle_new.drop(columns=['SALE DATE'], inplace=True)

/Users/patrickrue/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
# update borough codes to borough names 
# 1 = MANHATTAN
#  2 = BRONX
#  3 = BROOKLYN
#  4 = QUEENS
#  5 = STATEN ISLAND 

df_kaggle_new.loc[(df_kaggle_new['BOROUGH'] == 1),'BOROUGH'] ='MANHATTAN'
df_kaggle_new.loc[(df_kaggle_new['BOROUGH'] == 2),'BOROUGH'] = 'BRONX'
df_kaggle_new.loc[(df_kaggle_new['BOROUGH'] == 3),'BOROUGH'] = 'BROOKLYN'
df_kaggle_new.loc[(df_kaggle_new['BOROUGH'] == 4),'BOROUGH'] = 'QUEENS'
df_kaggle_new.loc[(df_kaggle_new['BOROUGH'] == 5),'BOROUGH'] = 'STATEN ISLAND'

/Users/patrickrue/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [26]:
df_kaggle_new.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,MANHATTAN,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899,1800000
3,MANHATTAN,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,125 AVENUE D,10009,2185,5725,1910,426000
11,MANHATTAN,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,14 AVENUE A,10009,2377,8722,1900,1600000
12,MANHATTAN,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,536 EAST 6 STREET,10009,1957,6200,1920,1060000
15,MANHATTAN,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,228 AVENUE B,10009,2842,12613,1900,3200000


In [27]:
df_kaggle_new.shape

(802833, 9)

In [28]:
# dropping duplicate columns 
df_kaggle_new.drop_duplicates(inplace=True)

/Users/patrickrue/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [29]:
df_kaggle_new.shape

(784210, 9)

In [30]:
# filtering the dataset for only family dwelling house categories 

df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("WALKUP APARTMENTS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("STORE BUILDINGS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("COMMERCIAL GARAGES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("CULTURAL FACILITIES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("WAREHOUSES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("OFFICE BUILDINGS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("ELEVATOR APARTMENTS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("LOFT BUILDINGS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("HOTEL")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("FACILITIES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("TAX CLASS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("FACTORIES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("ASYLUMS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("CULTURAL")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("THEATRES")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("BILLING")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("COMMERCIAL")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("PARKING")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("CONDOPS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("RENTALS")==False]
df_kaggle_new = df_kaggle_new[df_kaggle_new["BUILDING CLASS CATEGORY"].str.contains("CONDO COOPS")==False]

In [31]:
df_kaggle_new.shape

(389173, 9)

In [32]:
display(df_kaggle_new.head(7))

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
0,MANHATTAN,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899,1800000
125,MANHATTAN,ALPHABET CITY,15 CONDOS - 2-10 UNIT RESIDENTIAL,305 EAST 4 STREET,10009,0,0,0,400000
130,MANHATTAN,ALPHABET CITY,15 CONDOS - 2-10 UNIT RESIDENTIAL,366 EAST 8 STREET,10009,0,0,0,912500
134,MANHATTAN,ALPHABET CITY,15 CONDOS - 2-10 UNIT RESIDENTIAL,645 EAST 11 STREET,10009,0,0,0,390000
166,MANHATTAN,CHELSEA,02 TWO FAMILY HOMES,362 WEST 19 STREET,10011,1393,2340,1930,1500000
167,MANHATTAN,CHELSEA,02 TWO FAMILY HOMES,338 WEST 23 STREET,10011,2469,5400,1901,3400000
170,MANHATTAN,CHELSEA,03 THREE FAMILY HOMES,355 WEST 19 STREET,10011,1564,3264,1910,2525000


In [33]:
df_kaggle_new.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'ADDRESS',
       'ZIP CODE', 'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'SALE PRICE'],
      dtype='object')

In [34]:
# dropping rows with land sqr feet  less than 750 
df_kaggle_new = df_kaggle_new[df_kaggle_new['LAND SQUARE FEET'] > 750]

In [35]:
# dropping rows with gross sqr feet  less than 750 
df_kaggle_new = df_kaggle_new[df_kaggle_new['GROSS SQUARE FEET'] > 750]

In [36]:
# dropping rows with year built less than 1940
df_kaggle_new = df_kaggle_new[df_kaggle_new['YEAR BUILT'] > 1940]

In [37]:
# remove and row with zip code as zero 
df_kaggle_new = df_kaggle_new[df_kaggle_new['ZIP CODE'] > 0]

In [38]:
df_kaggle_new['YEAR BUILT'].min()

1941

In [39]:
df_kaggle_new['YEAR BUILT'].max()

2019

In [40]:
df_kaggle_new['ZIP CODE'].min()

10001

In [41]:
df_kaggle_new['ZIP CODE'].max()

11697

In [42]:
display(df_kaggle_new)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
2338,MANHATTAN,GRAMERCY,02 TWO FAMILY HOMES,349 2 AVENUE,10010,1580,11620,2009,1150000
4456,MANHATTAN,HARLEM-CENTRAL,02 TWO FAMILY HOMES,213 WEST 121 STREET,10027,2523,4600,2002,502693
4459,MANHATTAN,HARLEM-CENTRAL,02 TWO FAMILY HOMES,267 WEST 122 STREET,10027,2018,3680,2002,466157
4478,MANHATTAN,HARLEM-CENTRAL,03 THREE FAMILY HOMES,55 EAST 118 STREET,10035,2018,4000,2001,550755
4479,MANHATTAN,HARLEM-CENTRAL,03 THREE FAMILY HOMES,63 EAST 118 STREET,10035,2018,4000,2001,550755
...,...,...,...,...,...,...,...,...,...
1600191,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,33 QUAIL LANE,10309,2255,2377,1998,640000
1600192,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,22 HERRICK AVENUE,10309,4000,2400,2000,917000
1600194,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,62 AMBASSADOR LA,10309,2340,2160,2001,610000
1600195,STATEN ISLAND,WOODROW,02 TWO FAMILY DWELLINGS,104 AMBASSADOR LANE,10309,2730,2565,2001,625000


In [43]:
df_kaggle_new["BUILDING CLASS CATEGORY"].unique()

array(['02  TWO FAMILY HOMES                        ',
       '03  THREE FAMILY HOMES                      ',
       '01  ONE FAMILY HOMES                        ',
       '15  CONDOS - 2-10 UNIT RESIDENTIAL          ',
       '03  THREE FAMILY DWELLINGS                  ',
       '01  ONE FAMILY DWELLINGS                    ',
       '02  TWO FAMILY DWELLINGS                    ',
       '03 THREE FAMILY DWELLINGS                  ',
       '02 TWO FAMILY DWELLINGS                    ',
       '01 ONE FAMILY DWELLINGS                    ',
       '01 ONE FAMILY DWELLINGS', '15 CONDOS - 2-10 UNIT RESIDENTIAL',
       '02 TWO FAMILY DWELLINGS', '03 THREE FAMILY DWELLINGS'],
      dtype=object)

In [44]:
df_kaggle_new.shape

(151236, 9)

In [45]:
df_kaggle_new.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'ADDRESS',
       'ZIP CODE', 'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'SALE PRICE'],
      dtype='object')

In [46]:
# updating column names
df_kaggle_new.columns = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CATEGORY', 'ADDRESS', \
       'ZIP_CODE', 'LAND_SQR_FT', 'GROSS_SQR_FT', 'YEAR_BUILT', 'PRICE']

In [47]:
df_kaggle_new.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CATEGORY,ADDRESS,ZIP_CODE,LAND_SQR_FT,GROSS_SQR_FT,YEAR_BUILT,PRICE
2338,MANHATTAN,GRAMERCY,02 TWO FAMILY HOMES,349 2 AVENUE,10010,1580,11620,2009,1150000
4456,MANHATTAN,HARLEM-CENTRAL,02 TWO FAMILY HOMES,213 WEST 121 STREET,10027,2523,4600,2002,502693


In [48]:
df_kaggle_new = df_kaggle_new.reset_index(drop=True)

In [49]:
df_kaggle_new.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CATEGORY,ADDRESS,ZIP_CODE,LAND_SQR_FT,GROSS_SQR_FT,YEAR_BUILT,PRICE
0,MANHATTAN,GRAMERCY,02 TWO FAMILY HOMES,349 2 AVENUE,10010,1580,11620,2009,1150000
1,MANHATTAN,HARLEM-CENTRAL,02 TWO FAMILY HOMES,213 WEST 121 STREET,10027,2523,4600,2002,502693
2,MANHATTAN,HARLEM-CENTRAL,02 TWO FAMILY HOMES,267 WEST 122 STREET,10027,2018,3680,2002,466157
3,MANHATTAN,HARLEM-CENTRAL,03 THREE FAMILY HOMES,55 EAST 118 STREET,10035,2018,4000,2001,550755
4,MANHATTAN,HARLEM-CENTRAL,03 THREE FAMILY HOMES,63 EAST 118 STREET,10035,2018,4000,2001,550755


In [50]:
# df_kaggle_new = df_kaggle_new.sample(n = 20000)

In [51]:
df_kaggle_new.shape

(151236, 9)

In [52]:
# randomly select 50,000 rows
df_kaggle_new = df_kaggle_new.sample(n=2000)

In [53]:
df_kaggle_new.shape

(2000, 9)

# Extract Longitude and latitude from an API and assign the values to the dataframe for each of the addresses


https://developer.myptv.com/Documentation/Geocoding%20API/Code%20Samples/Locations%20by%20Address.htm

In [54]:
# create longitude and latitude columns 
df_kaggle_new['long'] = 'longitude'
df_kaggle_new['lat'] = 'lattitude'
df_kaggle_new.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CATEGORY,ADDRESS,ZIP_CODE,LAND_SQR_FT,GROSS_SQR_FT,YEAR_BUILT,PRICE,long,lat
24844,BROOKLYN,CANARSIE,02 TWO FAMILY HOMES,1065 EAST 104TH STREET,11236,2000,1635,1955,470000,longitude,lattitude
83000,QUEENS,ASTORIA,02 TWO FAMILY HOMES,21-13 43RD STREET,11105,1800,1816,1965,515000,longitude,lattitude
136743,QUEENS,FLUSHING-NORTH,03 THREE FAMILY DWELLINGS,45-33 160TH STREET,11358,3000,3040,1972,1640000,longitude,lattitude
32521,STATEN ISLAND,ANNADALE,01 ONE FAMILY HOMES,153 NORTH PINE TERRACE,10312,7840,1912,1965,540000,longitude,lattitude
131510,STATEN ISLAND,OAKWOOD-BEACH,01 ONE FAMILY DWELLINGS,342 FAIRBANKS AVENUE,10306,2300,1152,1970,560000,longitude,lattitude


In [55]:
address = list(df_kaggle_new['ADDRESS'])
locality = list(df_kaggle_new['BOROUGH'])
zipcode = list(df_kaggle_new['ZIP_CODE'])
new_list = list(zip(address, locality, zipcode ))

In [56]:
new_list[0]

('1065 EAST 104TH STREET                   ', 'BROOKLYN', 11236)

In [57]:
# create the url api variables 
# extract the long and lattittude 
# assign the long and lattitude for the addresses 
for i in range(len(new_list)):
    house_address = new_list[i][0].strip()
    house_number = new_list[i][0].split(" ")[0]
    address = " ".join(new_list[i][0].split(" ")[1:]).strip()
    country = "US"
    zipcode = new_list[i][2]
    state = "NEW YORK"
    locality = new_list[i][1]
    # assign variables to the url 
    url = f"https://api.myptv.com/geocoding/v1/locations/by-address?country={country}&locality={locality}&postalCode={zipcode} \
    &street={address}&houseNumber={house_number}&countryFilter={country}"
    headers = {'apiKey': "OWExZjY5NWU2MTc5NDU1ZWFkMmJkYTkxN2FmMWU5MjA6OTI3ZDNhMTctY2Y2OC00NmFjLWI4ZjMtNzdjMTk1N2EyYjQ2"}
    
    # response as json object 
    response = requests.request("GET", url, headers=headers).json()
    # create default dict to hold long and lat info
    long_lat  = defaultdict(list)
    # loop thru each response json object and parse the data 
    for i in range(len(response['locations'])):
        postal_code = response['locations'][i]['address']['postalCode'].split("-")[0]
        city = response['locations'][i]['address']['city']
        state = response['locations'][i]['address']['state']
        if postal_code == f"{zipcode}"  and state == 'New York':
            long_lat['long_ref'].append(response['locations'][i]['referencePosition']['longitude'])
            long_lat['lat_ref'].append(response['locations'][i]['referencePosition']['latitude'])
            
    # update the long, and lat columns
    # sometimes the long_lat dict is empty set default to 0 then 
    try:
        df_kaggle_new.loc[(df_kaggle_new['ADDRESS'].str.contains(f"{house_address}")), "long"] = max(long_lat['long_ref'])
        df_kaggle_new.loc[(df_kaggle_new['ADDRESS'].str.contains(f"{house_address}")), "lat"] = max(long_lat['lat_ref'])
        
    except ValueError:
        df_kaggle_new.loc[(df_kaggle_new['ADDRESS'].str.contains(f"{house_address}")), "long"] = 0
        df_kaggle_new.loc[(df_kaggle_new['ADDRESS'].str.contains(f"{house_address}")), "lat"] = 0
        

In [58]:
df_kaggle_new.shape

(2000, 11)

In [146]:
df_kaggle_new.loc[(df_kaggle_new['BUILDING_CATEGORY'].str.contains('02')), 'BUILDING_CATEGORY'] = 'TWO FAMILY HOMES'

In [147]:
df_kaggle_new.loc[(df_kaggle_new['BUILDING_CATEGORY'].str.contains('ONE')), 'BUILDING_CATEGORY'] = 'ONE FAMILY HOME'

In [148]:
df_kaggle_new.loc[(df_kaggle_new['BUILDING_CATEGORY'].str.contains('03 ')), 'BUILDING_CATEGORY'] = 'THREE FAMILY HOMES'

In [149]:
df_kaggle_new = df_kaggle_new[df_kaggle_new['long'] != 0]

In [150]:
df_kaggle_new.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING_CATEGORY,ADDRESS,ZIP_CODE,LAND_SQR_FT,GROSS_SQR_FT,YEAR_BUILT,PRICE,long,lat
24844,BROOKLYN,CANARSIE,TWO FAMILY HOMES,1065 EAST 104TH STREET,11236,2000,1635,1955,470000,-73.892036,40.644272
83000,QUEENS,ASTORIA,TWO FAMILY HOMES,21-13 43RD STREET,11105,1800,1816,1965,515000,-73.901871,40.77264


In [151]:
# write cleaned data to csv 
df_kaggle_new.to_csv('Resources/cleaned_nyc_housing_data.csv', index=False)

# Houses for Sales in the Boroughs selected.

In [190]:
# read the the selected borough data

df_houses = pd.read_csv(Path("Resources/housing_selected.csv"))

In [191]:
df_houses.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CATEGORY,ADDRESS,ZIP_CODE,LAND_SQR_FT,GROSS_SQR_FT,YEAR_BUILT,PRICE,long,lat
0,BROOKLYN,WILLIAMSBURG-NORTH,ONE FAMILY HOME,83 GRAND STREET,11249,1331,4032,2017,4450000,-73.964195,40.715740
1,BROOKLYN,PARK SLOPE,ONE FAMILY HOME,566 CARROLL STREET,11215,1143,3007,2015,3325000,-73.983429,40.675770
2,BROOKLYN,BROOKLYN HEIGHTS,ONE FAMILY HOME,5 COLUMBIA PLACE,11201,1241,3000,2003,3162500,-73.998306,40.693481
3,BROOKLYN,FLATBUSH-CENTRAL,ONE FAMILY HOME,1216 ALBEMARLE ROAD,11218,4000,2263,1960,2625000,-73.967682,40.646290
4,BROOKLYN,PARK SLOPE,15 CONDOS - 2-10 UNIT RESIDENTIAL,701 UNION STREET,11215,2375,1641,2013,2421500,-73.981560,40.676979


In [192]:
df_houses_new = df_houses.copy()

In [193]:
df_houses_new['PRICE'] = df_houses_new['PRICE']/10000

In [194]:
import hvplot.pandas
houses = df_houses_new.hvplot.points(
    'long', 
    'lat', 
    geo=True, 
    size='PRICE',
    color='BOROUGH',
    # alpha=0.2,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500
    ).opts(
    title = "Houses for sale in the Boroughs selected"
)
# Show the plot
houses

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [long,lat]   (BOROUGH,PRICE)